## 「（続）ChatGPTちゃんと日本語述語項構造解析で戯れる」のための類似文検索結果作り

 - このnotebookではfew-shot-semantic-search プロンプとのための類似文検索結果を作成していきます。
 - KWDLC のdevデータと類似するtrainデータを紐づけておきます
 - その紐づいたtrainデータを用いてfew-shot sampleなプロンプトを作っていこうという塩梅

In [ ]:
!pip install sentence_transformers
!pip install simpleneighbors annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 998.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1d4b5de476805da1dd1abcf344f8ce7902e58495e63b26e32314615cfc1ad318
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552449 sha256=d84022595446c62ef8dee711f94e232d698f43f21d0933f2705a8d34617e46ea
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
# google drive をマウント
# My Driveにアクセスできるようになる。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/')

In [ ]:
# 環境変数の準備
# openai のAPIキーをセット
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
my_secret = userdata.get("OPENAI_API_KEY")

# データのロードと整形

KWDLCのWebコーパスをダウンロードして、Googleドライブに格納しています。
https://github.com/ku-nlp/KWDLC

## 前処理

- train/dev/testのIDの取得
- orgの取得、idと文のlistで保持
- knpフォルダ下の関係を取得

In [ ]:
# train/dev/testのIDの取得

import glob
import re
import os
import pprint as pp

import json

# json_open = open('./sample_data/sample2.json', 'r')
json_open = open('/content/drive/My Drive/data/KWDLC/train_test_id.json', 'r')
train_test_id_dict = json.load(json_open)

# json_open = open('./sample_data/sample2.json', 'r')
json_open = open('/content/drive/My Drive/data/KWDLC/org.json', 'r')
org_dict = json.load(json_open)


In [ ]:
def ret_simple_org(org_dict):
  rdict = {}
  for id, sub_dict in org_dict.items():
    rdict[id] = ret_simple_sentence(sub_dict)
  return rdict

def ret_simple_sentence(sub_dict):
  rstr = ''
  rlist = []
  for sid, sentence in sub_dict.items():
    rlist.append(sentence)
  rstr = ''.join(rlist)
  return rstr

In [ ]:
train_sen_dict = {}
train_sen2id_dict = {}
train_str_list = []
for phase in ['train']:
  for id1 in train_test_id_dict[phase]:
    senstr = ret_simple_sentence(org_dict[id1])
    train_sen_dict[id1] = senstr
    if not senstr in train_sen2id_dict:
      train_sen2id_dict[senstr] = []
    train_sen2id_dict[senstr].append(id1)
# sen2id_dict

In [ ]:
from sentence_transformers import util, SentenceTransformer
import pandas as pd
import torch

# model_name = "intfloat/multilingual-e5-small"
# model_name = "intfloat/multilingual-e5-base"
model_name = "intfloat/multilingual-e5-large"

# モデルの読み込み
model = SentenceTransformer(model_name)

sentences = [{'id': 1, 'text': 'これはサンプル'}, {'id': 2, 'text': 'これはサンプル'}]
query = [{'id': 1, 'text': 'これはサンプル'}, {'id': 2, 'text': 'これはサンプル'}]

df = pd.DataFrame(sentences)
df["vector"] = df["text"].apply(model.encode)
df['vector'][0].shape

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

(1024,)

In [ ]:
from sentence_transformers import SentenceTransformer, util
from simpleneighbors import SimpleNeighbors

def find_model_with_name(models, name):
    for model in models:
        if model["name"] == name:
            return model
    raise NameError(f"Could not find model {name}.")

class SemanticSearch:
    def __init__(self, model):
        self.encoder = SentenceTransformer(model["name"])
        self.index = SimpleNeighbors(model["dims"], model["metric"])
        if model["metric"] == "angular":
            self.metric_func = util.cos_sim
        elif model["metric"] == "dot":
            self.metric_func = util.dot_score

    def load_corpus(self, filename):
        with open(f"corpus/{filename}") as f:
            self.feed(f.read().split("\n"))

    def set_corpus(self, sen_dict):
        for id1, sentence in sen_dict.items():
          vector = self.encoder.encode(sentence)
          self.index.add_one(sentence, vector)
        self.index.build()

    def feed(self, sentences):
        for sentence in sentences:
            vector = self.encoder.encode(sentence)
            print([sentence, vector.shape])
            self.index.add_one(sentence, vector)
        self.index.build()

    def find_nearest(self, query, n=5):
        vector = self.encoder.encode(query)
        nearests = self.index.nearest(vector, n)
        res = []
        for neighbor in nearests:
            dist = self.metric_func(vector, self.index.vec(neighbor))
            res.append((neighbor, float(dist)))
        return res

In [ ]:
models = [
    {
        # Multi-lingual model of Universal Sentence Encoder for 50 languages.
        "name": "intfloat/multilingual-e5-small",
        "dims": 384,
        "metric": "angular",
    },
    {
        # Multi-lingual model of Universal Sentence Encoder for 50 languages.
        "name": "intfloat/multilingual-e5-base",
        "dims": 768,
        "metric": "angular",
    },
        {
        # Multi-lingual model of Universal Sentence Encoder for 50 languages.
        "name": "intfloat/multilingual-e5-large",
        "dims": 1024,
        "metric": "angular",
    },
    {
        # Multi-lingual model of Universal Sentence Encoder for 15 languages:
        # Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish.
        "name": "distiluse-base-multilingual-cased-v1",
        "dims": 512,
        "metric": "angular",
    },
    {
        # Multi-lingual model of Universal Sentence Encoder for 50 languages.
        "name": "distiluse-base-multilingual-cased-v2",
        "dims": 512,
        "metric": "angular",
    },

]

In [ ]:
def save_result_json(result_dict, base_dir, model_name):

  import json
  import datetime

  # ファイル名を作成します。
  filename = f"{model_name}.json"

  # JSON形式に変換する際に、インデントと文字コードを指定します
  json_data = json.dumps(result_dict, indent=4, ensure_ascii=False)

  # チェックしたいディレクトリのパスを指定
  directory_path_to_check = base_dir + f'/KWDLC/pool/'

  # ディレクトリが存在しない場合は作成する
  create_directory_if_not_exists(directory_path_to_check)

  output_file_path = f'{directory_path_to_check}/{filename}'
  # JSONデータをファイルに保存します
  with open(output_file_path, 'w', encoding='utf-8') as f:
    print(f"saving {output_file_path}")
    f.write(json_data)
    # pass

def create_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"ディレクトリ {directory_path} を作成しました。")
    else:
        print(f"ディレクトリ {directory_path} はすでにあります。")

In [ ]:
print([len(org_dict), len(train_sen_dict), len(train_sen2id_dict)])
for senstr, array in train_sen2id_dict.items():
  if len(array) > 1:
    print(senstr, array)

[5127, 3915, 3913]
２００９年上半期は大麻の摘発が過去最多。覚せい剤の摘発は減少した。芸能人やミュージシャンの逮捕が相次ぐ。 ['w201106-0001077249', 'w201106-0001924496']
ハピ猫には現在１６匹の猫スタッフがみなさまのご来店をお待ちしています。猫スタッフは体調管理のため、ローテーションでお店に出ています。日によってお休みの子もいますのでご了承くださいませ。 ['w201106-0001695233', 'w201106-0001949558']


In [ ]:

model_name = "intfloat/multilingual-e5-large"
# model_name = "intfloat/multilingual-e5-small"
model = find_model_with_name(models, model_name)
ss = SemanticSearch(model)
# sample_dict = {'1': 'これはサンプルの文です。繰り返すこれはサンプルの文です。', '2': 'やめるにゃー！！'}
# ss.set_corpus(sample_dict)
ss.set_corpus(train_sen_dict)
print('index ok')

result_dict = {}
phase = "dev"
count = 0
for id1 in train_test_id_dict[phase]:
    senstr = ret_simple_sentence(org_dict[id1])
    res_list = []
    response = ss.find_nearest(senstr)
    for res in response:
      tstr = res[0]
      score = res[1]
      tid_list = []
      if tstr in train_sen2id_dict:
        tid_list = train_sen2id_dict[tstr]
      tmp_dict = {'score': score, 'train_str': tstr, 'id_list': tid_list}
      res_list.append(tmp_dict)
    result_dict[id1] = {'query': senstr, 'response': res_list}
    count += 1
    if count % 10 == 0 and count > 1:
      print([f"GoGo! {id1} {count}"])

BASE_DIR = '/content/drive/My Drive/data/'
exp_name = 'ss'
save_result_json(result_dict, BASE_DIR, exp_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]